# A* algorithm

In [113]:
# Import necessary libraries
import numpy as np
import pandas as pd
from heapq import heappush, heappop
import time
import math

In [114]:
# Read data from the CSV file into a Pandas DataFrame
df = pd.read_csv("~/Downloads/Flight_Data.csv")

In [115]:
class airport_information:
    def __init__(self, city, country, latitude, longitude, altitude):
        self.city = city
        self.country = country
        self.latitude = latitude
        self.longitude = longitude
        self.altitude = altitude

In [116]:
airports = set()
airports_name_to_index = dict()
airports_index_to_name = np.empty(4000, dtype=object)
airports_information = np.empty(4000, dtype=airport_information)

ind = 0
for i in df.index:
    airport = df["SourceAirport"][i]
    if airport not in airports:
        airports.add(airport)
        if df["SourceAirport_Country"][i] == "Dubai":
            print(airport)
        airports_name_to_index[airport] = ind
        airports_index_to_name[ind] = airport
        inf = airport_information(df["SourceAirport_City"][i], df["SourceAirport_Country"][i],
                                   df["SourceAirport_Latitude"][i], df["SourceAirport_Longitude"][i],
                                   df["SourceAirport_Altitude"][i])
        airports_information[ind] = inf
        ind += 1
        
    airport = df["DestinationAirport"][i]
    if airport not in airports:
        airports.add(airport)
        airports_name_to_index[airport] = ind
        airports_index_to_name[ind] = airport
        inf = airport_information(df["DestinationAirport_City"][i], df["DestinationAirport_Country"][i],
                                   df["DestinationAirport_Latitude"][i], df["DestinationAirport_Longitude"][i],
                                   df["DestinationAirport_Altitude"][i])
        airports_information[ind] = inf
        ind += 1

number_of_airports = len(airports)
print(number_of_airports)

759


In [117]:
def calc(distance, flytime, price):
    result = (1 * distance) + (80 * flytime) + (1 * price)
    return result
    
class flight_information:
    def __init__(self, source_airport, destination_airport, distance, flytime, price):
        self.source_airport = source_airport
        self.destination_airport = destination_airport
        self.distance = distance 
        self.flytime = flytime
        self.price = price
        self.cost = calc(distance, flytime, price)
        
class flights_list:
    def __init__(self):
        self.flights = []
    def add_flight(self, flight):
        self.flights.append(flight)

class ghf:
    def __init__(self, g, h):
        self.g = g
        self.h = h
        if self.g == math.inf or self.h == math.inf:
            self.f = math.inf
        else:
            self.f = g+h
    
    def set_g(self, g):
        self.g = g
        if self.g == math.inf:
            self.f = math.inf
        else:
            self.f = self.g + self.h

    def set_h(self, h):
        self.h = h
        if self.h == math.inf:
            self.f = math.inf
        else:
            self.f = self.g + self.h


In [118]:
flights_information = np.empty(4000, dtype=flights_list)
for i in range(number_of_airports):
    flights_information[i] = flights_list()

for i in df.index:
    index_source_airport = airports_name_to_index[df["SourceAirport"][i]]
    index_destination_airport = airports_name_to_index[df["DestinationAirport"][i]]
    distance = df["Distance"][i]
    flytime = df["FlyTime"][i]
    price = df["Price"][i]
    if distance < 0 or flytime < 0 or price < 0:
        continue
    inf = flight_information(index_source_airport, index_destination_airport, distance, flytime, price)
    flights_information[index_source_airport].add_flight(inf)

In [119]:
# inp = input()
inp = "Imam Khomeini International Airport - Raleigh Durham International Airport"
source_airport, destination_airport = inp.split('-')
source_airport = source_airport.strip()
destination_airport = destination_airport.strip()
index_source_airport = airports_name_to_index[source_airport]
index_destination_airport = airports_name_to_index[destination_airport]

print(index_source_airport, index_destination_airport)

1 80


In [120]:
def heuristic(cur_airport, destination):
    # Euclidean
    x = math.fabs (airports_information[cur_airport].latitude - airports_information[destination].latitude)
    y = math.fabs (airports_information[cur_airport].longitude - airports_information[destination].longitude)
    z = math.fabs (airports_information[cur_airport].altitude - airports_information[destination].altitude)
    
    return ((x*x) + (y*y) + (z*z)) ** 0.5

In [121]:
start_time = time.time()

openList = []
closedList = np.empty(4000, dtype=bool)
closedList.fill(False)
costs = np.empty(4000, dtype=ghf)
costs.fill(ghf(math.inf, math.inf))
path = np.empty(4000, dtype=int)
path.fill(-1)
cost_detail = np.empty(4000, dtype=flight_information)
cost_inf = flight_information(-1, -1, -1, -1, -1)
cost_detail.fill(cost_inf)

# Add values of start airport to the openList and
# set values of g, h, f and parent for start airport in this order
costs[index_source_airport].set_g(0)

for airport in range(number_of_airports):
    costs[airport].set_h(heuristic(airport, index_destination_airport))

heappush (openList, (costs[index_source_airport].f, index_source_airport))
found_destination = False

# Continue looping until the heap is not empty
while openList:
    # Get and remove the node with largest f
    cost, cur_airport = heappop(openList)
    closedList[cur_airport] = True

    for flight in flights_information[cur_airport].flights:
        if flight.destination_airport == index_destination_airport:
            found_destination = True
            break
        if not closedList[flight.destination_airport]:
            destination_g = costs[cur_airport].g + flight.cost
            destination_h = costs[flight.destination_airport].h
            destination_f = destination_g + destination_h

            print(destination_f, costs[flight.destination_airport].f)
            if costs[flight.destination_airport].f > destination_f:
                print("mamad")
                costs[flight.destination_airport].set_g = destination_g
                heappush(openList, (destination_f, flight.destination_airport))
                path[flight.destination_airport] = cur_airport
                cost_detail[flight.destination_airport] = flight

    if found_destination:
        break

total_time = round(time.time() - start_time, 6)

3599.5686789618717 325.2702707671356
3599.5686789618717 325.2702707671356
3599.5686789618717 325.2702707671356
3599.5686789618717 325.2702707671356
3599.5686789618717 325.2702707671356
2249.5702878968386 325.2702707671356
2361.173836883266 325.2702707671356
2254.306721300427 325.2702707671356
2254.306721300427 325.2702707671356
2254.306721300427 325.2702707671356
2254.306721300427 325.2702707671356
3531.695074078955 325.2702707671356
3531.695074078955 325.2702707671356
3016.023309018896 325.2702707671356
3016.023309018896 325.2702707671356
2140.486822062876 325.2702707671356


In [122]:
destination = index_destination_airport
way = []
total_distance = 0
total_flytime = 0
total_price = 0
r = 2 #round_value

while destination != -1:
    way.append(cost_detail[destination])
    destination = path[destination]
way.pop()
way = way[::-1]

In [123]:
print("A* Algorithm")
print(f"Execution Time: {round(total_time, 6)} Seconds")
print(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")
if len(way):
    for index, flight in enumerate(way):
        print(f"Flight #{index+1}:")
        print(f"From: {airports_information[flight.source_airport].city}-{airports_index_to_name[flight.source_airport]}, {airports_information[flight.source_airport].country}")
        print(f"To: {airports_information[flight.destination_airport].city}-{airports_index_to_name[flight.destination_airport]}, {airports_information[flight.destination_airport].country}")
        print(f"Duration: {round(flight.distance, r)} km")
        print(f"Time: {round(flight.flytime, r)} h")
        print(f"Price: {round(flight.price, r)} $")
        print("----------------------------")
        total_distance += flight.distance
        total_flytime += flight.flytime
        total_price += flight.price
        
    print(f"Total Price: {round(total_price, r)} $")
    print(f"Total Duration: {round(total_distance, r)} km")
    print(f"Total Time: {round(total_flytime, r)} h")
else:
    print("Way Not Found!")

A* Algorithm
Execution Time: 0.001766 Seconds
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Way Not Found!


In [124]:
f = open("1-UIAI4021-PR1-Q1-1.txt", "w")
f.write("Dijkstra Algorithm\n")
f.write(f"Execution Time: {round(total_time, 6)} Seconds\n")
f.write(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n")
if len(way):
    for index, flight in enumerate(way):
        f.write(f"Flight #{index+1}:\n")
        f.write(f"From: {airports_information[flight.source_airport].city}-{airports_index_to_name[flight.source_airport]}, {airports_information[flight.source_airport].country}\n")
        f.write(f"To: {airports_information[flight.destination_airport].city}-{airports_index_to_name[flight.destination_airport]}, {airports_information[flight.destination_airport].country}\n")
        f.write(f"Duration: {round(flight.distance, r)} km\n")
        f.write(f"Time: {round(flight.flytime, r)} h\n")
        f.write(f"Price: {round(flight.price, r)} $\n")
        f.write("----------------------------\n")
        
    f.write(f"Total Price: {round(total_price, r)} $\n")
    f.write(f"Total Duration: {round(total_distance, r)} km\n")
    f.write(f"Total Time: {round(total_flytime, r)} h\n")
else:
    f.write("Way Not Found!\n")
f.close()

## Dijkstra

In [125]:
start_time = time.time()

costs = np.empty(4000, dtype=float)
costs.fill(math.inf)
path = np.empty(4000, dtype=int)
path.fill(-1)
cost_detail = np.empty(4000, dtype=flight_information)
cost_inf = flight_information(-1, -1, -1, -1, -1)
cost_detail.fill(cost_inf)

open_list = []
heappush(open_list, (0, index_source_airport))
costs[index_source_airport] = 0

while open_list:
    cost, airport = heappop(open_list)
    if cost != costs[airport]: 
        continue
    for flight in flights_information[airport].flights:
        if flight.cost + cost < costs[flight.destination_airport]:
            costs[flight.destination_airport] = flight.cost + cost
            heappush(open_list, (costs[flight.destination_airport], flight.destination_airport))
            path[flight.destination_airport] = airport
            cost_detail[flight.destination_airport] = flight

total_time = round(time.time() - start_time, 6)
print(costs[index_destination_airport])
print(total_time)

17270.06657526321
0.009885


In [126]:
destination = index_destination_airport
way = []
total_distance = 0
total_flytime = 0
total_price = 0
r = 2 #round_value

while destination != -1:
    way.append(cost_detail[destination])
    destination = path[destination]
way.pop()
way = way[::-1]

In [127]:

    
print("Dijkstra Algorithm")
print(f"Execution Time: {round(total_time, 6)} Seconds")
print(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")
if len(way):
    for index, flight in enumerate(way):
        print(f"Flight #{index+1}:")
        print(f"From: {airports_information[flight.source_airport].city}-{airports_index_to_name[flight.source_airport]}, {airports_information[flight.source_airport].country}")
        print(f"To: {airports_information[flight.destination_airport].city}-{airports_index_to_name[flight.destination_airport]}, {airports_information[flight.destination_airport].country}")
        print(f"Duration: {round(flight.distance, r)} km")
        print(f"Time: {round(flight.flytime, r)} h")
        print(f"Price: {round(flight.price, r)} $")
        print("----------------------------")
        total_distance += flight.distance
        total_flytime += flight.flytime
        total_price += flight.price
        
    print(f"Total Price: {round(total_price, r)} $")
    print(f"Total Duration: {round(total_distance, r)} km")
    print(f"Total Time: {round(total_flytime, r)} h")
else:
    print("Way Not Found!")

Dijkstra Algorithm
Execution Time: 0.009885 Seconds
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1:
From: Tehran-Imam Khomeini International Airport, Iran
To: Istanbul-Atatürk International Airport, Turkey
Duration: 2040.98 km
Time: 3.04 h
Price: 990.49 $
----------------------------
Flight #2:
From: Istanbul-Atatürk International Airport, Turkey
To: New York-John F Kennedy International Airport, United States
Duration: 8051.74 km
Time: 10.5 h
Price: 3995.87 $
----------------------------
Flight #3:
From: New York-John F Kennedy International Airport, United States
To: Raleigh-durham-Raleigh Durham International Airport, United States
Duration: 686.5 km
Time: 1.35 h
Price: 313.25 $
----------------------------
Total Price: 5299.61 $
Total Duration: 10779.23 km
Total Time: 14.89 h


In [128]:
f = open("1-UIAI4021-PR1-Q1.txt", "w")
f.write("Dijkstra Algorithm\n")
f.write(f"Execution Time: {round(total_time, 6)} Seconds\n")
f.write(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n")
if len(way):
    for index, flight in enumerate(way):
        f.write(f"Flight #{index+1}:\n")
        f.write(f"From: {airports_information[flight.source_airport].city}-{airports_index_to_name[flight.source_airport]}, {airports_information[flight.source_airport].country}\n")
        f.write(f"To: {airports_information[flight.destination_airport].city}-{airports_index_to_name[flight.destination_airport]}, {airports_information[flight.destination_airport].country}\n")
        f.write(f"Duration: {round(flight.distance, r)} km\n")
        f.write(f"Time: {round(flight.flytime, r)} h\n")
        f.write(f"Price: {round(flight.price, r)} $\n")
        f.write("----------------------------\n")
        
    f.write(f"Total Price: {round(total_price, r)} $\n")
    f.write(f"Total Duration: {round(total_distance, r)} km\n")
    f.write(f"Total Time: {round(total_flytime, r)} h\n")
else:
    f.write("Way Not Found!\n")
f.close()